In [270]:
import pandas as pd

data = pd.read_stata('PS3_data.dta')

## Explore Data

We only have Heads of House Hold in our data

In [271]:
set(data.relhh.values)

{'Head'}

But, we have both males and females

In [272]:
set(data.hsex.values)

{1.0, 2.0}

We have only race = 1, 2, 3, 8, nan

In [273]:
data.hrace.unique() # note that we have no hispanics in our data, meaning a coefficient on hispanic dummy variable cannot be estimated.

array([  1.,   2.,   3.,  nan,   8.])

We also have individuals who worked 0 hours

In [274]:
min(set(data.hannhrs.values))

0.0

## Clean the data

In [275]:
#dropping female heads of house hold
data = data[data.hsex == 1]

#dropping individuals younger than 25, or older than 60
data = data[data.age <= 60]
data = data[data.age >= 25]

#dropping individuals with nan hours or annual income
data = data.dropna(how='any', subset=['hannhrs', 'hlabinc'])

#dropping individuals with 0 annual hours worked (to avoid +inf hourly wage)
data = data[data.hannhrs >0]

#dropping individuals with race, educ = nan
data = data.dropna(how='any', subset=['hrace', 'hyrsed'])



In [276]:
#create hourly wage variable = annual income/annual hours
data['hwage'] = data.hlabinc/data.hannhrs

#dropping individuals with wage <= 7 
data = data[data.hwage > 7]

#creating race dummies

import numpy as np

data['black'] = np.where(data['hrace']==2, 1, 0)

data['hispanic'] = np.where(data['hrace']==5, 1, 0)

data['other'] = np.where(((data['hrace'] !=1 ) & (data['hrace'] != 2) & (data['hrace'] !=5)), 1, 0)

## Check the variables we just created:

In [277]:
print(set(data.hsex.values), 'Should be ==1 only')
print(min(set(data.hannhrs.values)), 'Hours should be non-zero')
print(min(set(data.hwage.values)), max(set(data.hwage.values)), 'wage should be >7 and finite')
print(min(set(data.age.values)), max(set(data.age.values)), 'age should be [25, 60]')
print(min(set(data.hyrsed.values)), max(set(data.hyrsed.values)), 'edu range')

{1.0} Should be ==1 only
2.0 Hours should be non-zero
7.00025 1717.33 wage should be >7 and finite
25.0 60.0 age should be [25, 60]
1.0 17.0 edu range


In [278]:
print(set(data.hrace.values), 'only have race = 1, 2, 3')

print(len(data.hrace[data.hrace == 2]), sum(data.black), 'These must match')

print(len(data.hrace[data.hrace == 5]), sum(data.hispanic), 'These must match')

print(len(data.hrace[(data.hrace != 1) & (data.hrace !=2) & (data.hrace != 5) ]), sum(data.other), 'These must match')

white = len(data) - sum(data.black) - sum(data.hispanic) - sum(data.other)
print(len(data.hrace[data.hrace == 1]), white, 'These must match')

print(len(data)) 

{1.0, 2.0, 3.0} only have race = 1, 2, 3
3217 3217 These must match
0 0 These must match
1285 1285 These must match
52560 52560 These must match
57062


In [279]:
#transforming wage into logs:

data['loghwage'] = np.log(data.hwage)

## Removing unecessary data

In [280]:
model_data = data.loc[0:,['id68', 'year', 'loghwage', 'hyrsed', 'age', 'black',  'hispanic', 'other']]

## negative log liklihood function

In [281]:
def norm_LL(B, sd, data, y, x, t):
    '''
    This function returns the negative Log Likelihood of a linear model where observations are i.i.d. with 
    variance s^2 given parameters.
    
    Args:
        B: a (kx1) array of slope coefficients of the model 
        
        sd: the standard deviation of the error term in the model
        
        data: a pandas dataframe containing the model data in a panel format (individual over time)
        
        y: a str, which is the outcome variable of the model matching the dataframe column name
        
        
        x: a list of lenght k, the elements of which are the independent variables of the model matchin the dataframe column names
    
        t: the time period(s) of interest. a list of length m which reflects the time of interest from the panel.
    '''
    
    import numpy as np
    
    use_data = data[data['year'].isin(t)]
    
    outcome = np.matrix(use_data[list(y)])
    
    ind = np.matrix(use_data[x])
    
    n = len(use_data)
    
    B = np.matrix(B).T
    
    #negative log likilihood for use in optimization
    LL = -1*((n*np.log(2*np.pi) - n*np.log(sd) - (1/sd))*(1/2) - np.dot((outcome - ind*B).T, (outcome - ind*B)))
    
    return LL

## Test the log liklihood function with arbitrary betas

In [282]:
B = np.array((.5,.5,.5,0,.5))


In [283]:
sd = 1

y = ['loghwage']

x = ['hyrsed', 'age', 'black', 'other']

t = [1986]

params = (sd, model_data, y, x, t)

In [284]:
result = norm_LL(B, sd, model_data, y, x, t)
print(result)

ValueError: shapes (1992,4) and (5,1) not aligned: 4 (dim 1) != 5 (dim 0)

In [ ]:
import scipy.optimize as opt

beta = opt.minimize(norm_LL, B, args = params, method = 'Nelder-Mead', tol = 1e-15, options={'maxiter': 5000})

In [ ]:
beta

In [ ]:
use_data = model_data[model_data['year'] == 1986]

In [ ]:
ind = np.matrix(use_data[['hyrsed', 'age', 'black', 'other']])
outcome = np.matrix(use_data[y])

In [ ]:
np.linalg.inv((ind.T*ind))*ind.T*outcome

In [ ]:
set(use_data['hispanic'])=={0}

In [ ]:
x

In [ ]:
x.remove('hispanic')


In [ ]:
x